In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!ls -R /kaggle/input/

In [ ]:
# """
# NFL Big Data Bowl 2026 - Kaggle Inference (Feedforward Version)
# Predict future player positions using FeedforwardTimeModel.
# """

# import os
# import warnings
# from pathlib import Path
# import numpy as np
# import pandas as pd
# import polars as pl
# import torch
# import torch.nn as nn
# import joblib

# warnings.filterwarnings("ignore")

# # ============================================================================
# # CONFIG
# # ============================================================================
# class Config:
#     DATA_DIR = Path("/kaggle/input/nfl-big-data-bowl-2026-prediction/")
#     SEED = 42
#     N_FOLDS = 5
#     WINDOW_SIZE = 8
#     HIDDEN_DIM = 128
#     MAX_FUTURE_HORIZON = 94
#     FIELD_X_MIN, FIELD_X_MAX = 0.0, 120.0
#     FIELD_Y_MIN, FIELD_Y_MAX = 0.0, 53.3
#     DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Update this to your model folder (or use auto-detect below)
# import glob
# candidates = sorted(glob.glob("/kaggle/input/*/model_x_fold1.pth"))
# if candidates:
#     MODEL_ROOT = Path(candidates[0]).parent
#     print(f"✅ Auto-detected model folder: {MODEL_ROOT}")
# else:
#     MODEL_ROOT = Path("/kaggle/input/epoch200/model_save/epoch200")
#     print(f"⚠️ Default MODEL_ROOT = {MODEL_ROOT}")

# def set_seed(seed=42):
#     import random
#     random.seed(seed)
#     np.random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed_all(seed)
# set_seed(Config.SEED)

# # ============================================================================
# # MODEL
# # ============================================================================
# class FeedforwardTimeModel(nn.Module):
#     def __init__(self, in_dim):
#         super().__init__()
#         self.net = nn.Sequential(
#             nn.Linear(in_dim, 128),
#             nn.ReLU(),
#             nn.Linear(128, 128),
#             nn.ReLU(),
#             nn.Linear(128, 2)  # predict x,y displacements
#         )
#     def forward(self, x):
#         return self.net(x)

# # ============================================================================
# # FEATURE ENGINEERING (reuses same logic, last frame only)
# # ============================================================================
# def _height_to_feet_array(series: pd.Series) -> np.ndarray:
#     arr = np.asarray(series)
#     out = np.full(arr.shape, 6.0, dtype=np.float32)
#     for i, v in enumerate(arr):
#         try:
#             if isinstance(v, str) and "-" in v:
#                 ft, inch = v.split("-", 1)
#                 ft = int(ft.strip()) if ft.strip().isdigit() else 6
#                 inch = int(inch.strip()) if inch.strip().isdigit() else 0
#                 out[i] = ft + inch / 12.0
#             elif isinstance(v, (int, float)) and not np.isnan(v):
#                 out[i] = float(v)
#         except Exception:
#             continue
#     return out

# def prepare_last_frame_features(df: pd.DataFrame, feature_cols=None):
#     df = df.copy()
#     for c in ["player_height", "player_side", "player_role", "player_weight", "dir"]:
#         if c not in df.columns:
#             df[c] = np.nan

#     df["player_height_feet"] = _height_to_feet_array(df["player_height"])
#     dir_rad = np.deg2rad(df["dir"].fillna(0.0).to_numpy(dtype=np.float32))
#     s = df["s"].fillna(0.0).to_numpy(dtype=np.float32)
#     a = df["a"].fillna(0.0).to_numpy(dtype=np.float32)
#     df["velocity_x"] = s * np.sin(dir_rad)
#     df["velocity_y"] = s * np.cos(dir_rad)
#     df["acceleration_x"] = a * np.sin(dir_rad)
#     df["acceleration_y"] = a * np.cos(dir_rad)
#     ps = df["player_side"].astype(str).fillna("")
#     pr = df["player_role"].astype(str).fillna("")
#     df["is_offense"] = (ps == "Offense").astype(np.int8)
#     df["is_defense"] = (ps == "Defense").astype(np.int8)
#     df["is_receiver"] = (pr == "Targeted Receiver").astype(np.int8)
#     df["is_coverage"] = (pr == "Defensive Coverage").astype(np.int8)
#     df["is_passer"] = (pr == "Passer").astype(np.int8)
#     df["momentum_x"] = df["velocity_x"] * (df["player_weight"].fillna(200.0)/2.20462)
#     df["momentum_y"] = df["velocity_y"] * (df["player_weight"].fillna(200.0)/2.20462)
#     df["kinetic_energy"] = 0.5 * (df["player_weight"].fillna(200.0)/2.20462) * (df["s"].fillna(0.0)**2)

#     # Choose last frame per (game_id, play_id, nfl_id)
#     gcols = ["game_id","play_id","nfl_id"]
#     last_df = df.sort_values(["game_id","play_id","nfl_id","frame_id"]).groupby(gcols).tail(1)
    
#     if feature_cols is None:
#         feature_cols = [
#             "x","y","s","a","o","dir","player_height_feet","player_weight",
#             "velocity_x","velocity_y","acceleration_x","acceleration_y",
#             "momentum_x","momentum_y","kinetic_energy",
#             "is_offense","is_defense","is_receiver","is_coverage","is_passer"
#         ]
#         feature_cols = [c for c in feature_cols if c in last_df.columns]

#     X = last_df[feature_cols].fillna(0.0).to_numpy(dtype=np.float32)
#     meta = last_df[gcols + ["frame_id"]]
#     return X, meta, feature_cols

# # ============================================================================
# # PREDICT FUNCTION (for Kaggle)
# # ============================================================================
# def predict(test: pl.DataFrame, test_input: pl.DataFrame) -> pl.DataFrame:
#     test_pd = test.to_pandas()
#     test_input_pd = test_input.to_pandas()

#     # Prepare features
#     X, meta, feature_cols = prepare_last_frame_features(test_input_pd)
#     input_dim = X.shape[1]

#     # Load model/scaler folds
#     model_x_paths = [MODEL_ROOT / f"model_x_fold{i+1}.pth" for i in range(Config.N_FOLDS)]
#     model_y_paths = [MODEL_ROOT / f"model_y_fold{i+1}.pth" for i in range(Config.N_FOLDS)]
#     scaler_paths  = [MODEL_ROOT / f"scaler_fold{i+1}.pkl" for i in range(Config.N_FOLDS)]

#     for p in model_x_paths + model_y_paths + scaler_paths:
#         if not p.exists():
#             raise FileNotFoundError(f"Missing model/scaler file: {p}")

#     models, scalers = [], []
#     for i in range(Config.N_FOLDS):
#         scaler = joblib.load(scaler_paths[i])
#         scalers.append(scaler)
#         model = FeedforwardTimeModel(in_dim=input_dim).to(Config.DEVICE)
#         state = torch.load(model_x_paths[i], map_location="cpu")
#         if isinstance(state, dict) and "state_dict" in state:
#             state = state["state_dict"]
#         model.load_state_dict(state, strict=False)
#         model.eval()
#         models.append(model)

#     # Scale inputs
#     X_scaled = np.stack([scalers[0].transform(X)]).squeeze(0)
#     X_tensor = torch.tensor(X_scaled, dtype=torch.float32).to(Config.DEVICE)

#     # Predict ensemble
#     preds = []
#     with torch.no_grad():
#         for model in models:
#             preds.append(model(X_tensor).cpu().numpy())
#     preds = np.mean(preds, axis=0)  # [N,2]

#     pred_x, pred_y = preds[:,0], preds[:,1]
#     pred_x = np.clip(pred_x, Config.FIELD_X_MIN, Config.FIELD_X_MAX)
#     pred_y = np.clip(pred_y, Config.FIELD_Y_MIN, Config.FIELD_Y_MAX)

#     result = pl.DataFrame({"x": pred_x, "y": pred_y})
#     return result

# # ============================================================================
# # Kaggle Inference Server
# # ============================================================================
# try:
#     from kaggle_evaluation.nfl_inference_server import NFLInferenceServer
# except ImportError:
#     raise ImportError("Please run this in Kaggle's NFL inference environment")

# inference_server = NFLInferenceServer(predict)
# if os.getenv("KAGGLE_IS_COMPETITION_RERUN"):
#     inference_server.serve()
# else:
#     inference_server.run_local_gateway((str(Config.DATA_DIR),))
# print("Completed")

In [ ]:
# """
# NFL Big Data Bowl 2026 - Kaggle Inference (patched robust version)
# Self-contained predict() for Kaggle evaluation server.
# """

# import os
# import warnings
# from pathlib import Path
# import numpy as np
# import pandas as pd
# import polars as pl
# import torch
# import torch.nn as nn
# import joblib

# warnings.filterwarnings("ignore")

# # ============================================================================
# # CONFIG (self-contained)
# # ============================================================================
# class Config:
#     DATA_DIR = Path("/kaggle/input/nfl-big-data-bowl-2026-prediction/")
#     SEED = 42
#     N_FOLDS = 5
#     BATCH_SIZE = 512
#     WINDOW_SIZE = 8
#     HIDDEN_DIM = 128
#     MAX_FUTURE_HORIZON = 94
#     FIELD_X_MIN, FIELD_X_MAX = 0.0, 120.0
#     FIELD_Y_MIN, FIELD_Y_MAX = 0.0, 53.3
#     DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Change this to match your uploaded model folder in /kaggle/input/
# #MODEL_ROOT = Path("/kaggle/input/nfl-epoch200/model_save/epoch200")
# MODEL_ROOT = Path("/kaggle/input/epoch200/model_save/epoch200")


# def set_seed(seed=42):
#     import random
#     random.seed(seed)
#     np.random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed_all(seed)
#     os.environ["PYTHONHASHSEED"] = str(seed)

# set_seed(Config.SEED)

# # ============================================================================
# # FEATURE ENGINEERING (robust to type errors)
# # ============================================================================
# def _height_to_feet_array(series: pd.Series) -> np.ndarray:
#     """
#     Robust converter for strings like '6-2' -> 6.1667 feet.
#     Handles NaN, numbers, and non-string gracefully.
#     """
#     arr = np.asarray(series)
#     out = np.full(arr.shape, 6.0, dtype=np.float32)
#     for i, v in enumerate(arr):
#         try:
#             if isinstance(v, str) and "-" in v:
#                 ft, inch = v.split("-", 1)
#                 ft = int(ft.strip()) if ft.strip().isdigit() else 6
#                 inch = int(inch.strip()) if inch.strip().isdigit() else 0
#                 out[i] = ft + inch / 12.0
#             elif isinstance(v, (int, float)) and not np.isnan(v):
#                 out[i] = float(v)
#         except Exception:
#             continue
#     return out

# def prepare_sequences_fixed(input_df: pd.DataFrame,
#                             output_df: pd.DataFrame | None = None,
#                             test_template: pd.DataFrame | None = None,
#                             is_training: bool = True,
#                             window_size: int = 8,
#                             feature_cols_forced: list[str] | None = None):
#     df = input_df.copy()

#     # Ensure required identifier columns exist
#     for c in ["game_id","play_id","nfl_id","frame_id"]:
#         if c not in df.columns:
#             raise KeyError(f"Missing required column in test_input: {c}")

#     # Add missing numeric columns
#     needed = [
#         "x","y","s","a","o","dir",
#         "player_height","player_weight","player_side","player_role",
#         "ball_land_x","ball_land_y",
#     ]
#     for c in needed:
#         if c not in df.columns:
#             df[c] = np.nan

#     # Height
#     df["player_height_feet"] = _height_to_feet_array(df["player_height"])

#     # Direction
#     dir_rad = np.deg2rad(df["dir"].fillna(0.0).to_numpy(dtype=np.float32))
#     delta_t = 0.1
#     s = df["s"].fillna(0.0).to_numpy(dtype=np.float32)
#     a = df["a"].fillna(0.0).to_numpy(dtype=np.float32)
#     df["velocity_x"] = (s + 0.5 * a * delta_t) * np.sin(dir_rad)
#     df["velocity_y"] = (s + 0.5 * a * delta_t) * np.cos(dir_rad)
#     df["acceleration_x"] = a * np.sin(dir_rad)
#     df["acceleration_y"] = a * np.cos(dir_rad)

#     # Roles (robust string casting)
#     ps = df["player_side"].astype(str).fillna("")
#     pr = df["player_role"].astype(str).fillna("")
#     df["is_offense"]  = (ps == "Offense").astype(np.int8)
#     df["is_defense"]  = (ps == "Defense").astype(np.int8)
#     df["is_receiver"] = (pr == "Targeted Receiver").astype(np.int8)
#     df["is_coverage"] = (pr == "Defensive Coverage").astype(np.int8)
#     df["is_passer"]   = (pr == "Passer").astype(np.int8)

#     # Physics
#     mass_kg = df["player_weight"].fillna(200.0).to_numpy(dtype=np.float32) / 2.20462
#     df["momentum_x"] = df["velocity_x"] * mass_kg
#     df["momentum_y"] = df["velocity_y"] * mass_kg
#     df["kinetic_energy"] = 0.5 * mass_kg * (df["s"].fillna(0.0).to_numpy(dtype=np.float32) ** 2)

#     # Ball features
#     if "ball_land_x" in df.columns and "ball_land_y" in df.columns:
#         ball_dx = df["ball_land_x"].to_numpy(dtype=np.float32) - df["x"].to_numpy(dtype=np.float32)
#         ball_dy = df["ball_land_y"].to_numpy(dtype=np.float32) - df["y"].to_numpy(dtype=np.float32)
#         dist = np.sqrt(ball_dx**2 + ball_dy**2)
#         df["distance_to_ball"] = dist
#         df["angle_to_ball"] = np.arctan2(ball_dy, ball_dx)
#         df["ball_direction_x"] = ball_dx / (dist + 1e-6)
#         df["ball_direction_y"] = ball_dy / (dist + 1e-6)
#         df["closing_speed"] = (
#             df["velocity_x"].to_numpy(dtype=np.float32) * df["ball_direction_x"].to_numpy(dtype=np.float32) +
#             df["velocity_y"].to_numpy(dtype=np.float32) * df["ball_direction_y"].to_numpy(dtype=np.float32)
#         )
#     else:
#         df["distance_to_ball"] = np.nan
#         df["angle_to_ball"] = np.nan
#         df["ball_direction_x"] = np.nan
#         df["ball_direction_y"] = np.nan
#         df["closing_speed"] = np.nan

#     # Sort
#     df = df.sort_values(["game_id","play_id","nfl_id","frame_id"]).reset_index(drop=True)
#     gcols = ["game_id","play_id","nfl_id"]

#     # Lag & EMA features
#     for lag in (1,2,3):
#         df[f"x_lag{lag}"] = df.groupby(gcols)["x"].shift(lag)
#         df[f"y_lag{lag}"] = df.groupby(gcols)["y"].shift(lag)
#         df[f"velocity_x_lag{lag}"] = df.groupby(gcols)["velocity_x"].shift(lag)
#         df[f"velocity_y_lag{lag}"] = df.groupby(gcols)["velocity_y"].shift(lag)

#     df["velocity_x_ema"] = df.groupby(gcols)["velocity_x"].transform(lambda x: x.ewm(alpha=0.3, adjust=False).mean())
#     df["velocity_y_ema"] = df.groupby(gcols)["velocity_y"].transform(lambda x: x.ewm(alpha=0.3, adjust=False).mean())
#     df["speed_ema"]      = df.groupby(gcols)["s"].transform(lambda x: x.ewm(alpha=0.3, adjust=False).mean())
#     df["velocity_x_roll"] = df.groupby(gcols)["velocity_x"].transform(lambda x: x.rolling(window_size, min_periods=1).mean())
#     df["velocity_y_roll"] = df.groupby(gcols)["velocity_y"].transform(lambda x: x.rolling(window_size, min_periods=1).mean())

#     default_feature_cols = [
#         "x","y","s","a","o","dir","frame_id",
#         "ball_land_x","ball_land_y",
#         "player_height_feet","player_weight",
#         "velocity_x","velocity_y","acceleration_x","acceleration_y",
#         "momentum_x","momentum_y","kinetic_energy",
#         "is_offense","is_defense","is_receiver","is_coverage","is_passer",
#         "distance_to_ball","angle_to_ball","ball_direction_x","ball_direction_y","closing_speed",
#         "x_lag1","y_lag1","velocity_x_lag1","velocity_y_lag1",
#         "x_lag2","y_lag2","velocity_x_lag2","velocity_y_lag2",
#         "x_lag3","y_lag3","velocity_x_lag3","velocity_y_lag3",
#         "velocity_x_ema","velocity_y_ema","speed_ema",
#         "velocity_x_roll","velocity_y_roll",
#     ]

#     if feature_cols_forced is not None:
#         feature_cols = [c for c in feature_cols_forced if c in df.columns]
#     else:
#         feature_cols = [c for c in default_feature_cols if c in df.columns]

#     df.set_index(gcols, inplace=True)
#     grouped = df.groupby(level=gcols)

#     assert not is_training
#     assert test_template is not None
#     target_groups = test_template[gcols].drop_duplicates()
#     sequences, sequence_ids, last_xy = [], [], []
#     for _, row in target_groups.iterrows():
#         key = (row["game_id"], row["play_id"], row["nfl_id"])
#         try:
#             gdf = grouped.get_group(key)
#         except KeyError:
#             continue
#         win = gdf.tail(window_size)
#         if len(win) < window_size:
#             pad_len = window_size - len(win)
#             pad_df = pd.DataFrame(np.nan, index=range(pad_len), columns=gdf.columns)
#             win = pd.concat([pad_df, win.reset_index(drop=True)], ignore_index=True)
#         win = win.fillna(gdf.mean(numeric_only=True)).fillna(0.0)
#         seq = win[feature_cols].to_numpy(dtype=np.float32)
#         sequences.append(seq)
#         last_xy.append((float(win.iloc[-1]["x"]), float(win.iloc[-1]["y"])))
#         sequence_ids.append({"game_id": key[0], "play_id": key[1], "nfl_id": key[2],
#                              "frame_id": int(win.iloc[-1]["frame_id"])})
#     return sequences, sequence_ids, last_xy, feature_cols

# # ============================================================================
# # MODEL
# # ============================================================================
# class ImprovedSeqModel(nn.Module):
#     def __init__(self, input_dim, horizon):
#         super().__init__()
#         self.horizon = horizon
#         self.gru = nn.GRU(input_dim, 128, num_layers=2, batch_first=True, dropout=0.1)
#         self.pool_ln = nn.LayerNorm(128)
#         self.pool_attn = nn.MultiheadAttention(128, num_heads=4, batch_first=True)
#         self.pool_query = nn.Parameter(torch.randn(1, 1, 128))
#         self.head = nn.Sequential(
#             nn.Linear(128, 128),
#             nn.GELU(),
#             nn.Dropout(0.2),
#             nn.Linear(128, horizon),
#         )

#     def forward(self, x):
#         h, _ = self.gru(x)
#         B = h.size(0)
#         q = self.pool_query.expand(B, 1, -1)
#         h_norm = self.pool_ln(h)
#         ctx, _ = self.pool_attn(q, h_norm, h_norm)
#         ctx = ctx.squeeze(1)
#         out = self.head(ctx)
#         out = torch.cumsum(out, dim=1)
#         return out

# def _remap_state_dict_keys(sd: dict) -> dict:
#     new_sd = {}
#     for k, v in sd.items():
#         nk = k.replace("pool_queries", "pool_query") if "pool_queries" in k else k
#         new_sd[nk] = v
#     return new_sd

# # ============================================================================
# # INFERENCE: predict()
# # ============================================================================
# def predict(test: pl.DataFrame, test_input: pl.DataFrame) -> pl.DataFrame:
#     test_pd = test.to_pandas()
#     test_input_pd = test_input.to_pandas()
#     feature_cols_forced = None 
#     MODEL_ROOT = Path("/kaggle/input/epoch200/model_save/epoch200")
#     feat_path = MODEL_ROOT / "feature_cols.pkl"
#     if feat_path.exists():
#         try:
#             feature_cols_forced = joblib.load(feat_path)
#         except Exception:
#             feature_cols_forced = None

#     seqs, seq_ids, last_xy, feature_cols = prepare_sequences_fixed(
#         test_input_pd, test_template=test_pd, is_training=False,
#         window_size=Config.WINDOW_SIZE, feature_cols_forced=feature_cols_forced
#     )
#     if len(seqs) == 0:
#         return pl.DataFrame({"x": np.zeros(len(test_pd)), "y": np.zeros(len(test_pd))})

#     input_dim = seqs[0].shape[1]
#     model_x_paths = [MODEL_ROOT / f"model_x_fold{i+1}.pth" for i in range(Config.N_FOLDS)]
#     model_y_paths = [MODEL_ROOT / f"model_y_fold{i+1}.pth" for i in range(Config.N_FOLDS)]
#     scaler_paths  = [MODEL_ROOT / f"scaler_fold{i+1}.pkl"   for i in range(Config.N_FOLDS)]

#     for p in model_x_paths + model_y_paths + scaler_paths:
#         if not p.exists():
#             raise FileNotFoundError(f"Missing model/scaler file: {p}")

#     models_x, models_y, scalers = [], [], []
#     for i in range(Config.N_FOLDS):
#         sc = joblib.load(scaler_paths[i])
#         scalers.append(sc)
#         mx = ImprovedSeqModel(input_dim, Config.MAX_FUTURE_HORIZON).to(Config.DEVICE)
#         my = ImprovedSeqModel(input_dim, Config.MAX_FUTURE_HORIZON).to(Config.DEVICE)
#         sd_x = torch.load(model_x_paths[i], map_location="cpu")
#         sd_y = torch.load(model_y_paths[i], map_location="cpu")
#         if isinstance(sd_x, dict) and "state_dict" in sd_x: sd_x = sd_x["state_dict"]
#         if isinstance(sd_y, dict) and "state_dict" in sd_y: sd_y = sd_y["state_dict"]
#         mx.load_state_dict(_remap_state_dict_keys(sd_x), strict=False)
#         my.load_state_dict(_remap_state_dict_keys(sd_y), strict=False)
#         mx.eval(); my.eval()
#         models_x.append(mx); models_y.append(my)

#     X_scaled = np.stack([scalers[0].transform(s) for s in seqs]).astype(np.float32)
#     X_tensor = torch.from_numpy(X_scaled).to(Config.DEVICE)
#     with torch.no_grad():
#         all_dx, all_dy = [], []
#         for mx, my in zip(models_x, models_y):
#             dx = mx(X_tensor).cpu().numpy()
#             dy = my(X_tensor).cpu().numpy()
#             all_dx.append(dx); all_dy.append(dy)
#         ens_dx, ens_dy = np.mean(all_dx, axis=0), np.mean(all_dy, axis=0)

#     keycols = ["game_id","play_id","nfl_id"]
#     test_sorted = test_pd.reset_index(drop=True)
#     group_frames = (
#         test_sorted.groupby(keycols)["frame_id"]
#         .apply(lambda s: np.sort(s.to_numpy(dtype=np.int32)))
#         .to_dict()
#     )
#     preds, H = {}, ens_dx.shape[1]
#     for i, sid in enumerate(seq_ids):
#         key = (sid["game_id"], sid["play_id"], sid["nfl_id"])
#         frames = group_frames.get(key)
#         if frames is None: continue
#         x0, y0 = last_xy[i]
#         px = np.clip(x0 + ens_dx[i], Config.FIELD_X_MIN, Config.FIELD_X_MAX)
#         py = np.clip(y0 + ens_dy[i], Config.FIELD_Y_MIN, Config.FIELD_Y_MAX)
#         d = {int(fid): (float(px[min(t,H-1)]), float(py[min(t,H-1)])) for t, fid in enumerate(frames)}
#         preds[key] = d
#     out_x, out_y = np.empty(len(test_sorted), np.float32), np.empty(len(test_sorted), np.float32)
#     for idx, row in test_sorted.iterrows():
#         key = (row["game_id"], row["play_id"], row["nfl_id"])
#         fid = int(row["frame_id"])
#         if key in preds and fid in preds[key]:
#             x, y = preds[key][fid]
#         else:
#             x = float(row.get("x", 0.0) or 0.0)
#             y = float(row.get("y", 0.0) or 0.0)
#         out_x[idx], out_y[idx] = x, y
#     return pl.DataFrame({"x": out_x, "y": out_y})

# # ============================================================================
# # Kaggle Inference Server
# # ============================================================================
# try:
#     from kaggle_evaluation.nfl_inference_server import NFLInferenceServer
# except ImportError:
#     raise ImportError("Please run in Kaggle's NFL inference environment")

# inference_server = NFLInferenceServer(predict)
# if os.getenv("KAGGLE_IS_COMPETITION_RERUN"):
#     inference_server.serve()
# else:
#     inference_server.run_local_gateway((str(Config.DATA_DIR),))
# print("Completed")

In [ ]:
"""
NFL Big Data Bowl 2026 - Kaggle Inference (improved robust version)
Self-contained predict() for Kaggle evaluation server.
"""

import os
import warnings
from pathlib import Path
import numpy as np
import pandas as pd
import polars as pl
import torch
import torch.nn as nn
import joblib

warnings.filterwarnings("ignore")

# ============================================================================
# CONFIG
# ============================================================================
class Config:
    DATA_DIR = Path("/kaggle/input/nfl-big-data-bowl-2026-prediction/")
    SEED = 42
    N_FOLDS = 5
    BATCH_SIZE = 512
    WINDOW_SIZE = 8
    HIDDEN_DIM = 128
    MAX_FUTURE_HORIZON = 94
    FIELD_X_MIN, FIELD_X_MAX = 0.0, 120.0
    FIELD_Y_MIN, FIELD_Y_MAX = 0.0, 53.3
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

MODEL_ROOT = Path("/kaggle/input/epoch200/model_save/epoch200")

# Deterministic setup
def set_seed(seed=42):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(Config.SEED)

# ============================================================================
# FEATURE ENGINEERING
# ============================================================================
def _height_to_feet_array(series: pd.Series) -> np.ndarray:
    arr = np.asarray(series)
    out = np.full(arr.shape, 6.0, dtype=np.float32)
    for i, v in enumerate(arr):
        try:
            if isinstance(v, str) and "-" in v:
                ft, inch = v.split("-", 1)
                ft = int(ft.strip()) if ft.strip().isdigit() else 6
                inch = int(inch.strip()) if inch.strip().isdigit() else 0
                out[i] = ft + inch / 12.0
            elif isinstance(v, (int, float)) and not np.isnan(v):
                out[i] = float(v)
        except Exception:
            continue
    return out

def prepare_sequences_fixed(input_df: pd.DataFrame,
                            output_df: pd.DataFrame | None = None,
                            test_template: pd.DataFrame | None = None,
                            is_training: bool = True,
                            window_size: int = 8,
                            feature_cols_forced: list[str] | None = None):
    df = input_df.copy()
    for c in ["game_id","play_id","nfl_id","frame_id"]:
        if c not in df.columns:
            raise KeyError(f"Missing required column: {c}")

    needed = [
        "x","y","s","a","o","dir",
        "player_height","player_weight","player_side","player_role",
        "ball_land_x","ball_land_y",
    ]
    for c in needed:
        if c not in df.columns:
            df[c] = np.nan

    df["player_height_feet"] = _height_to_feet_array(df["player_height"])

    dir_rad = np.deg2rad(df["dir"].fillna(0.0).to_numpy(dtype=np.float32))
    delta_t = 0.1
    s = df["s"].fillna(0.0).to_numpy(dtype=np.float32)
    a = df["a"].fillna(0.0).to_numpy(dtype=np.float32)
    df["velocity_x"] = (s + 0.5 * a * delta_t) * np.sin(dir_rad)
    df["velocity_y"] = (s + 0.5 * a * delta_t) * np.cos(dir_rad)
    df["acceleration_x"] = a * np.sin(dir_rad)
    df["acceleration_y"] = a * np.cos(dir_rad)

    ps = df["player_side"].astype(str).fillna("")
    pr = df["player_role"].astype(str).fillna("")
    df["is_offense"]  = (ps == "Offense").astype(np.int8)
    df["is_defense"]  = (ps == "Defense").astype(np.int8)
    df["is_receiver"] = (pr == "Targeted Receiver").astype(np.int8)
    df["is_coverage"] = (pr == "Defensive Coverage").astype(np.int8)
    df["is_passer"]   = (pr == "Passer").astype(np.int8)

    mass_kg = df["player_weight"].fillna(200.0).to_numpy(dtype=np.float32) / 2.20462
    df["momentum_x"] = df["velocity_x"] * mass_kg
    df["momentum_y"] = df["velocity_y"] * mass_kg
    df["kinetic_energy"] = 0.5 * mass_kg * (df["s"].fillna(0.0).to_numpy(dtype=np.float32) ** 2)

    ball_dx = df["ball_land_x"].to_numpy(dtype=np.float32) - df["x"].to_numpy(dtype=np.float32)
    ball_dy = df["ball_land_y"].to_numpy(dtype=np.float32) - df["y"].to_numpy(dtype=np.float32)
    dist = np.sqrt(ball_dx**2 + ball_dy**2)
    df["distance_to_ball"] = dist
    df["angle_to_ball"] = np.arctan2(ball_dy, ball_dx)
    df["ball_direction_x"] = ball_dx / (dist + 1e-6)
    df["ball_direction_y"] = ball_dy / (dist + 1e-6)
    df["closing_speed"] = (
        df["velocity_x"] * df["ball_direction_x"] +
        df["velocity_y"] * df["ball_direction_y"]
    )

    df = df.sort_values(["game_id","play_id","nfl_id","frame_id"]).reset_index(drop=True)
    gcols = ["game_id","play_id","nfl_id"]

    for lag in (1,2,3):
        df[f"x_lag{lag}"] = df.groupby(gcols)["x"].shift(lag)
        df[f"y_lag{lag}"] = df.groupby(gcols)["y"].shift(lag)
        df[f"velocity_x_lag{lag}"] = df.groupby(gcols)["velocity_x"].shift(lag)
        df[f"velocity_y_lag{lag}"] = df.groupby(gcols)["velocity_y"].shift(lag)

    df["velocity_x_ema"] = df.groupby(gcols)["velocity_x"].transform(lambda x: x.ewm(alpha=0.3, adjust=False).mean())
    df["velocity_y_ema"] = df.groupby(gcols)["velocity_y"].transform(lambda x: x.ewm(alpha=0.3, adjust=False).mean())
    df["speed_ema"]      = df.groupby(gcols)["s"].transform(lambda x: x.ewm(alpha=0.3, adjust=False).mean())
    df["velocity_x_roll"] = df.groupby(gcols)["velocity_x"].transform(lambda x: x.rolling(window_size, min_periods=1).mean())
    df["velocity_y_roll"] = df.groupby(gcols)["velocity_y"].transform(lambda x: x.rolling(window_size, min_periods=1).mean())

    default_feature_cols = [
        "x","y","s","a","o","dir","frame_id",
        "ball_land_x","ball_land_y",
        "player_height_feet","player_weight",
        "velocity_x","velocity_y","acceleration_x","acceleration_y",
        "momentum_x","momentum_y","kinetic_energy",
        "is_offense","is_defense","is_receiver","is_coverage","is_passer",
        "distance_to_ball","angle_to_ball","ball_direction_x","ball_direction_y","closing_speed",
        "x_lag1","y_lag1","velocity_x_lag1","velocity_y_lag1",
        "x_lag2","y_lag2","velocity_x_lag2","velocity_y_lag2",
        "x_lag3","y_lag3","velocity_x_lag3","velocity_y_lag3",
        "velocity_x_ema","velocity_y_ema","speed_ema",
        "velocity_x_roll","velocity_y_roll",
    ]

    if feature_cols_forced is not None:
        missing = [c for c in feature_cols_forced if c not in df.columns]
        if missing:
            feature_cols_forced = None

    feature_cols = feature_cols_forced or [c for c in default_feature_cols if c in df.columns]

    df.set_index(gcols, inplace=True)
    grouped = df.groupby(level=gcols)

    assert not is_training
    assert test_template is not None
    target_groups = test_template[gcols].drop_duplicates()
    sequences, sequence_ids, last_xy = [], [], []
    for _, row in target_groups.iterrows():
        key = (row["game_id"], row["play_id"], row["nfl_id"])
        try:
            gdf = grouped.get_group(key)
        except KeyError:
            continue
        win = gdf.tail(window_size)
        if len(win) < window_size:
            pad_len = window_size - len(win)
            pad_df = pd.DataFrame(np.nan, index=range(pad_len), columns=gdf.columns)
            win = pd.concat([pad_df, win.reset_index(drop=True)], ignore_index=True)
        win = win.ffill().bfill().fillna(gdf.mean(numeric_only=True)).fillna(0.0)
        seq = win[feature_cols].to_numpy(dtype=np.float32)
        sequences.append(seq)
        last_xy.append((float(win.iloc[-1]["x"]), float(win.iloc[-1]["y"])))
        sequence_ids.append({"game_id": key[0], "play_id": key[1], "nfl_id": key[2],
                             "frame_id": int(win.iloc[-1]["frame_id"])})
    return sequences, sequence_ids, last_xy, feature_cols

# ============================================================================
# MODEL
# ============================================================================
class ImprovedSeqModel(nn.Module):
    def __init__(self, input_dim, horizon):
        super().__init__()
        self.horizon = horizon
        self.gru = nn.GRU(input_dim, 128, num_layers=2, batch_first=True, dropout=0.1)
        self.pool_ln = nn.LayerNorm(128)
        self.pool_attn = nn.MultiheadAttention(128, num_heads=4, batch_first=True)
        self.pool_query = nn.Parameter(torch.randn(1, 1, 128))
        self.head = nn.Sequential(
            nn.Linear(128, 128),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(128, horizon),
        )

    def forward(self, x):
        h, _ = self.gru(x)
        B = h.size(0)
        q = self.pool_query.expand(B, 1, -1)
        h_norm = self.pool_ln(h)
        ctx, _ = self.pool_attn(q, h_norm, h_norm)
        ctx = ctx.squeeze(1)
        out = self.head(ctx)
        out = torch.cumsum(out, dim=1)
        return out

def _remap_state_dict_keys(sd: dict) -> dict:
    new_sd = {}
    for k, v in sd.items():
        nk = k.replace("pool_queries", "pool_query") if "pool_queries" in k else k
        new_sd[nk] = v
    return new_sd

# ============================================================================
# INFERENCE
# ============================================================================
def _batched_forward(model, xt, batch_size):
    outs = []
    for b in range(0, xt.size(0), batch_size):
        xb = xt[b:b+batch_size]
        with torch.no_grad():
            outs.append(model(xb).cpu().numpy())
    return np.concatenate(outs, axis=0)

def predict(test: pl.DataFrame, test_input: pl.DataFrame) -> pl.DataFrame:
    test_pd = test.to_pandas()
    test_input_pd = test_input.to_pandas()
    feature_cols_forced = None

    feat_path = MODEL_ROOT / "feature_cols.pkl"
    if feat_path.exists():
        try:
            feature_cols_forced = joblib.load(feat_path)
        except Exception:
            feature_cols_forced = None

    seqs, seq_ids, last_xy, feature_cols = prepare_sequences_fixed(
        test_input_pd, test_template=test_pd, is_training=False,
        window_size=Config.WINDOW_SIZE, feature_cols_forced=feature_cols_forced
    )
    if len(seqs) == 0:
        return pl.DataFrame({"x": np.zeros(len(test_pd)), "y": np.zeros(len(test_pd))})

    input_dim = seqs[0].shape[1]
    model_x_paths = [MODEL_ROOT / f"model_x_fold{i+1}.pth" for i in range(Config.N_FOLDS)]
    model_y_paths = [MODEL_ROOT / f"model_y_fold{i+1}.pth" for i in range(Config.N_FOLDS)]
    scaler_paths  = [MODEL_ROOT / f"scaler_fold{i+1}.pkl"   for i in range(Config.N_FOLDS)]

    for p in model_x_paths + model_y_paths + scaler_paths:
        if not p.exists():
            raise FileNotFoundError(f"Missing model/scaler file: {p}")

    models_x, models_y, scalers = [], [], []
    for i in range(Config.N_FOLDS):
        sc = joblib.load(scaler_paths[i])
        scalers.append(sc)
        mx = ImprovedSeqModel(input_dim, Config.MAX_FUTURE_HORIZON).to(Config.DEVICE)
        my = ImprovedSeqModel(input_dim, Config.MAX_FUTURE_HORIZON).to(Config.DEVICE)
        sd_x = torch.load(model_x_paths[i], map_location="cpu")
        sd_y = torch.load(model_y_paths[i], map_location="cpu")
        if isinstance(sd_x, dict) and "state_dict" in sd_x: sd_x = sd_x["state_dict"]
        if isinstance(sd_y, dict) and "state_dict" in sd_y: sd_y = sd_y["state_dict"]
        mx.load_state_dict(_remap_state_dict_keys(sd_x), strict=False)
        my.load_state_dict(_remap_state_dict_keys(sd_y), strict=False)
        mx.eval(); my.eval()
        models_x.append(mx); models_y.append(my)

    # --- Per-fold scaling & batching ---
    with torch.no_grad():
        all_dx, all_dy = [], []
        for i, (mx, my, sc) in enumerate(zip(models_x, models_y, scalers)):
            X_scaled_i = np.stack([sc.transform(s) for s in seqs]).astype(np.float32)
            X_tensor_i = torch.from_numpy(X_scaled_i).to(Config.DEVICE)
            dx_i = _batched_forward(mx, X_tensor_i, Config.BATCH_SIZE)
            dy_i = _batched_forward(my, X_tensor_i, Config.BATCH_SIZE)
            all_dx.append(dx_i)
            all_dy.append(dy_i)
        ens_dx = np.mean(all_dx, axis=0)
        ens_dy = np.mean(all_dy, axis=0)

    keycols = ["game_id","play_id","nfl_id"]
    test_sorted = test_pd.reset_index(drop=True)
    group_frames = (
        test_sorted.groupby(keycols)["frame_id"]
        .apply(lambda s: np.sort(s.to_numpy(dtype=np.int32)))
        .to_dict()
    )
    preds, H = {}, ens_dx.shape[1]
    for i, sid in enumerate(seq_ids):
        key = (sid["game_id"], sid["play_id"], sid["nfl_id"])
        frames = group_frames.get(key)
        if frames is None: continue
        x0, y0 = last_xy[i]
        px = np.clip(x0 + ens_dx[i], Config.FIELD_X_MIN, Config.FIELD_X_MAX)
        py = np.clip(y0 + ens_dy[i], Config.FIELD_Y_MIN, Config.FIELD_Y_MAX)
        d = {int(fid): (float(px[min(t,H-1)]), float(py[min(t,H-1)])) for t, fid in enumerate(frames)}
        preds[key] = d

    out_x, out_y = np.empty(len(test_sorted), np.float32), np.empty(len(test_sorted), np.float32)
    for idx, row in test_sorted.iterrows():
        key = (row["game_id"], row["play_id"], row["nfl_id"])
        fid = int(row["frame_id"])
        if key in preds and fid in preds[key]:
            x, y = preds[key][fid]
        else:
            x = float(row.get("x", 0.0) or 0.0)
            y = float(row.get("y", 0.0) or 0.0)
        out_x[idx], out_y[idx] = x, y

    return pl.DataFrame({"x": out_x, "y": out_y})

# ============================================================================
# Kaggle Inference Server
# ============================================================================
try:
    from kaggle_evaluation.nfl_inference_server import NFLInferenceServer
except ImportError:
    raise ImportError("Please run in Kaggle's NFL inference environment")

inference_server = NFLInferenceServer(predict)
if os.getenv("KAGGLE_IS_COMPETITION_RERUN"):
    inference_server.serve()
else:
    inference_server.run_local_gateway((str(Config.DATA_DIR),))
print("✅ Inference completed successfully.")
